# Aide Jazmín González Cruz

In [ ]:
# https://pypi.org/project/sodapy/
# https://www.sustainabilist.com/blog/chicago-data-analysis-a-internship-project
# https://dev.socrata.com/consumers/getting-started.html

# https://holowczak.com/getting-started-with-nyc-opendata-and-the-socrata-api/5/

# https://dev.socrata.com/docs/functions/between.html
# https://pydigger.com/pypi/sodapy

# https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/data

# https://data.cityofchicago.org/resource/4ijn-s7e5.csv?$where=inspection_date>='2021-01-01' and inspection_date<='2021-01-07'

## Cargando librerías

In [1]:
import pandas as pd
from sodapy import Socrata
import pickle
from datetime import date, timedelta
import boto3
import yaml
# saber qué versión de Boto estamos ocupando
# print(boto3.__version__)

### Iniciando fechas

In [2]:
today = date.today()
print(today)
date_ini = date.today() - timedelta(7)
print(date_ini)
date_fin = date.today() - timedelta(1)
print(date_fin)

2021-02-20
2021-02-13
2021-02-19


In [7]:
today.strftime('%Y-%m-%d')

'2021-02-20'

# API Chicago

In [3]:
# The Host Name for the API endpoint (the https:// part will be added automatically)
data_url='data.cityofchicago.org'   
# The data set at the API endpoint
data_set='4ijn-s7e5'   
# The app token created in the prior steps
app_token='QcC2iXBnip0eXLBnriENWyee4'   
# Create the client to point to the API endpoint
client = Socrata(data_url,app_token)      
# Set the timeout to 60 seconds    
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=300000)
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
df.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2484862,"MEMO'S RESTAURANT,INC.",MEMO'S RESTAURANT,2428947,Restaurant,Risk 1 (High),6322 W GRAND AVE,CHICAGO,IL,60639,2021-02-18T00:00:00.000,Canvass Re-Inspection,Pass,41.922264833522355,-87.78383149141781,"{'latitude': '-87.78383149141781', 'longitude'...",NaN
1,2484868,CARNICERIA LA VILLA NO.2,CARNICERIA LA VILLA #2,2575130,Grocery Store,Risk 1 (High),5800 W GRAND AVE,CHICAGO,IL,60639,2021-02-18T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,41.9197749009712,-87.77072652582828,"{'latitude': '-87.77072652582828', 'longitude'...","48. WAREWASHING FACILITIES: INSTALLED, MAINTAI..."
2,2484890,WALMART NEIGHBORHOOD MARKET #3166,WALMART NEIGHBORHOOD MARKET #3166,2334925,Grocery Store,Risk 1 (High),4720 S COTTAGE GROVE AVE,CHICAGO,IL,60615,2021-02-18T00:00:00.000,Complaint,Pass,41.8089016636963,-87.60669696618142,"{'latitude': '-87.60669696618142', 'longitude'...","55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ..."
3,2484872,NICK'S ON NORTH,NICK'S ON NORTH,2670594,Restaurant,Risk 1 (High),3323 W NORTH AVE,CHICAGO,IL,60647,2021-02-18T00:00:00.000,Canvass Re-Inspection,Pass,41.90995972342911,-87.71042949751607,"{'latitude': '-87.71042949751607', 'longitude'...",NaN
4,2484869,"LITTLE HANDS LEARNING CENTER ACADEMY, INC","LITTLE HANDS LEARNING CENTER ACADEMY, INC",2216116,1023 CHILDERN'S SERVICES FACILITY,Risk 1 (High),10126 S WESTERN AVE,CHICAGO,IL,60643,2021-02-18T00:00:00.000,License,Pass,41.70921709772374,-87.68185356852545,"{'latitude': '-87.68185356852545', 'longitude'...",39. CONTAMINATION PREVENTED DURING FOOD PREPAR...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216458 entries, 0 to 216457
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   inspection_id    216458 non-null  object
 1   dba_name         216458 non-null  object
 2   aka_name         213971 non-null  object
 3   license_         216441 non-null  object
 4   facility_type    211570 non-null  object
 5   risk             216389 non-null  object
 6   address          216458 non-null  object
 7   city             216297 non-null  object
 8   state            216408 non-null  object
 9   zip              216412 non-null  object
 10  inspection_date  216458 non-null  object
 11  inspection_type  216457 non-null  object
 12  results          216458 non-null  object
 13  latitude         215742 non-null  object
 14  longitude        215742 non-null  object
 15  location         215742 non-null  object
 16  violations       158626 non-null  object
dtypes: object(

In [7]:
# Save the data frame to a CSV file
# df.to_csv("output/2020-02-02_d.csv")
#pickle.dump(df, open("output/2020-02-02_.pkl", "wb"))

# https://www.geeksforgeeks.org/understanding-python-pickling-example/
# Pickling without a file

# For storing 
bd_historica = pickle.dumps(df)       # type(b) gives <class 'bytes'> 
  
# For loading 
# myEntry = pickle.loads(b) 
# print(myEntry) 


In [14]:
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries

where_condition = "inspection_date >= '" + date_ini +"' and inspection_date <= '" + date_fin + "' "
print(where_condition)

results2 = client.get(data_set, 
                      where=where_condition, 
                      limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df2 = pd.DataFrame.from_records(results2)
df2.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2472253,CHITOWN SANDWICH CLUB,CHITOWN SANDWICH CLUB,2574541,Restaurant,Risk 1 (High),2921 N CLARK ST,CHICAGO,IL,60657,2021-01-04T00:00:00.000,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.935452917475686,-87.64686341693728,"{'latitude': '-87.64686341693728', 'longitude'..."
1,2472237,AFC SUSHI @ JEWEL OSCO#2502,AFC SUSHI @ JEWEL OSCO,1985397,Restaurant,Risk 1 (High),2550 N CLYBOURN,CHICAGO,IL,60614,2021-01-04T00:00:00.000,Canvass,Pass w/ Conditions,56. ADEQUATE VENTILATION & LIGHTING; DESIGNATE...,41.92787172311833,-87.67325881463049,"{'latitude': '-87.67325881463049', 'longitude'..."
2,2464207,MACHINE,MACHINE,2595542,Restaurant,Risk 1 (High),1836-1846 W DIVISION ST,CHICAGO,IL,60622,2021-01-04T00:00:00.000,Non-Inspection,No Entry,NaN,41.90335448822961,-87.67381134493849,"{'latitude': '-87.67381134493849', 'longitude'..."
3,2464210,FOURTH CHURCH DAY SCHOOL,FOURTH CHURCH DAY SCHOOL,2215767,DAYCARE,Risk 1 (High),126 E CHESTNUT ST,CHICAGO,IL,60611,2021-01-04T00:00:00.000,Canvass,Pass,NaN,41.89848890902573,-87.62455288409814,"{'latitude': '-87.62455288409814', 'longitude'..."
4,2402212,Zaytune Mediterranean Grill,Zaytune,2009543,Restaurant,Risk 1 (High),3129 S Morgan ST,CHICAGO,IL,60608,2021-01-04T00:00:00.000,Canvass,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.83713294787266,-87.65097503508115,"{'latitude': '-87.65097503508115', 'longitude'..."


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inspection_id    1413 non-null   object
 1   dba_name         1413 non-null   object
 2   aka_name         1409 non-null   object
 3   license_         1413 non-null   object
 4   facility_type    1408 non-null   object
 5   risk             1413 non-null   object
 6   address          1413 non-null   object
 7   city             1408 non-null   object
 8   state            1413 non-null   object
 9   zip              1412 non-null   object
 10  inspection_date  1413 non-null   object
 11  inspection_type  1413 non-null   object
 12  results          1413 non-null   object
 13  violations       993 non-null    object
 14  latitude         1406 non-null   object
 15  longitude        1406 non-null   object
 16  location         1406 non-null   object
dtypes: object(17)
memory usage: 187.8

In [16]:
# Save the data frame to a CSV file
# df2.to_csv("output/2021-01-01_d2.csv")
# pickle.dump(df2, open("output/2021-01-01_d2.pkl", "wb"))

In [21]:
fecha = dt.strftime("%Y-%m-%d")
where_cond = "inspection_date > '" + fecha + "' "
where_cond

"inspection_date > '2021-02-09' "

In [22]:
results3 = client.get(data_set, where=where_cond, limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df3 = pd.DataFrame.from_records(results3)
df3.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2484753,SUBWAY,SUBWAY,1985307,Restaurant,Risk 1 (High),4242 S WENTWORTH AVE,CHICAGO,IL,60609,2021-02-10T00:00:00.000,Canvass Re-Inspection,Pass,41.81698453270526,-87.6313645580596,"{'latitude': '-87.6313645580596', 'longitude':...",NaN
1,2484705,KOKO'S MEDITERRANEAN GRILL,KOKO'S MEDITERRANEAN GRILL,2293013,Restaurant,Risk 1 (High),1760 W CHICAGO AVE,CHICAGO,IL,60622,2021-02-10T00:00:00.000,Canvass,Fail,41.89610267284695,-87.67211594992528,"{'latitude': '-87.67211594992528', 'longitude'...",60. PREVIOUS CORE VIOLATION CORRECTED - Commen...
2,2484700,SUPERMERCADO EL TAXQUENITO,SUPERMERCADO EL TAXQUENITO,2240202,Grocery Store,Risk 1 (High),6309 W GRAND AVE,CHICAGO,IL,60639,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.92189422993173,-87.78339752005009,"{'latitude': '-87.78339752005009', 'longitude'...",NaN
3,2484743,ZANNO'S FISH & CHICKEN,ZANNO'S,2704295,Restaurant,Risk 1 (High),1521 S PULASKI RD,CHICAGO,IL,60623,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.86003909876977,-87.72484229344634,"{'latitude': '-87.72484229344634', 'longitude'...",NaN
4,2484741,CLUB NUTRICION LIA,CLUB NUTRICION LIA,2663532,Restaurant,Risk 2 (Medium),3720 W 26TH ST,CHICAGO,IL,60623,2021-02-10T00:00:00.000,Non-Inspection,No Entry,41.84444353922239,-87.71821191334412,"{'latitude': '-87.71821191334412', 'longitude'...",NaN


In [23]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inspection_id    27 non-null     object
 1   dba_name         27 non-null     object
 2   aka_name         27 non-null     object
 3   license_         27 non-null     object
 4   facility_type    27 non-null     object
 5   risk             27 non-null     object
 6   address          27 non-null     object
 7   city             27 non-null     object
 8   state            27 non-null     object
 9   zip              27 non-null     object
 10  inspection_date  27 non-null     object
 11  inspection_type  27 non-null     object
 12  results          27 non-null     object
 13  latitude         27 non-null     object
 14  longitude        27 non-null     object
 15  location         27 non-null     object
 16  violations       13 non-null     object
dtypes: object(17)
memory usage: 3.7+ KB


In [24]:
# Save the data frame to a CSV file
df3.to_csv("output/" + fecha + ".csv")
pickle.dump(df3, open("output/" + fecha + ".pkl", "wb"))

# AWS

## Funciones solicitadas

In [33]:
# ================================= src/utils/general.py =======================================
# este archivo yaml vive en tu conf/local/credentials.yaml
# este va en src/utils general.py
def read_yaml_file(yaml_file):
    """ 
    load yaml cofigurations 
    """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Could not load the file')
    
    return config

# este va en src/utils general.py
def get_s3_credentials(credentials_file):
    """ 
    Regresa credenciales de conexión a AWS 
    """
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds

def get_api_token(credentials_file):
    """ 
    Regresa token API
    """
    credentials = read_yaml_file(credentials_file)
    token = credentials['food_inspections']
    token = token["api_token"]

    return token
   
# ----------------------------------------------------------------------------------------------

# ============================ src/pipeline/ingesta_almacenamiento.py ==========================

def get_client():
    """ 
    Esta función regresa un cliente que se puede conectar a la API de inspecciones 
    de establecimiento dándole un token previamente generado.
    """    
    # "../../conf/local/credentials.yaml"
    token = get_api_token("credentials.yaml")
    
    # The Host Name for the API endpoint (the https:// part will be added automatically)
    data_url='data.cityofchicago.org'  
    # Create the client to point to the API endpoint, with app token created in the prior steps
    client = Socrata(data_url, token) 
    
    return client  


def ingesta_inicial(client, limite):
    """ 
    Esta función recibe como parámetros el cliente con el que nos podemos comunicar con la API, 
    y el límite de registros que queremos obtener al llamar a la API. 
    Regresa una lista de los elementos que la API regresó.
    """   
    # The data set at the API endpoint
    data_set='4ijn-s7e5' 
    # Set the timeout to 60 seconds    
    client.timeout = 60
    # Retrieve the first 'limite' results returned as JSON object from the API
    # The SoDaPy library converts this JSON object to a Python list of dictionaries
    results = client.get(data_set, where="inspection_date > '2020-01-02' ", limit = limite)#QUITAR FECHA****
    
    #df = pd.DataFrame.from_records(results)
    #df.head()
    #df.info()
    #bd_historica = pickle.dumps(df) 
    bd_historica = pickle.dumps(results) 
    
    return bd_historica


def get_s3_resource():     
    """ 
    Esta función regresa un resource de S3 para poder guardar datos 
    en el bucket (checar script de aws_boto_s3).
    """
    # "../../conf/local/credentials.yaml"
    s3_creds = get_s3_credentials("credentials.yaml")
    
    session = boto3.Session(
        aws_access_key_id=s3_creds['aws_access_key_id'],
        aws_secret_access_key=s3_creds['aws_secret_access_key']
    )
    
    s3 = session.resource('s3')
    #ses = boto3.session.Session(profile_name='aide_jgc', region_name='us-east-2')
    #s3 = ses.resource('s3')
    
    return s3
    
def guardar_ingesta(bucket, bucket_path, data):      
    """ 
    Esta función recibe como parámetros el nombre de tu bucket de S3, 
    la ruta en el bucket en donde se guardarán los datos y los datos ingestados en pkl.
    
    #  https://stackoverflow.com/questions/40336918/how-to-write-a-file-or-data-to-an-s3-object-using-boto3
    import boto3

    some_binary_data = b'Here we have some data'
    more_binary_data = b'Here we have some more data'

    # Method 1: Object.put()
    s3 = boto3.resource('s3')
    object = s3.Object('my_bucket_name', 'my/key/including/filename.txt')
    object.put(Body=some_binary_data)

    # Method 2: Client.put_object()
    client = boto3.client('s3')
    client.put_object(Body=more_binary_data, 
                      Bucket='my_bucket_name', 
                      Key='my/key/including/anotherfilename.txt')
    """
    s3_resource = get_s3_resource()
    
    today = date.today()
    # ingestion/initial/historic-inspections-2020-02-02.pkl
    # ingestion/consecutive/consecutive-inspections-2020-11-03.pkl
    file_name = bucket_path + today.strftime('%Y-%m-%d') + ".pkl"

    s3_resource.Object(bucket, file_name).put(Body = data)
    
    print("Ingesta guardada")
    
    
def ingesta_consecutiva(client, fecha, limite):
    """
    Esta función recibe como parámetros el cliente con el que nos podemos comunicar con la API, 
    la fecha de la que se quieren obtener nuevos datos al llamar a la API y 
    el límite de registros para obtener de regreso.
    """   
    # The data set at the API endpoint
    data_set='4ijn-s7e5' 

    # Set the timeout to 60 seconds    
    client.timeout = 60
    # fecha.strftime("%Y-%m-%d") --> en caso de formato fecha
    where_cond = "inspection_date >= '" + fecha + "' "
    results = client.get(data_set, where = where_cond, limit = limite)
    
    #df = pd.DataFrame.from_records(results)
    #return df
    
    bd_consecutiva = pickle.dumps(results) 
    
    return bd_consecutiva
    

***Listado de solo una carpeta en específico***

### Probando funciones

In [3]:
# get_api_token
token = get_api_token("credentials.yaml")
token

'QcC2iXBnip0eXLBnriENWyee4'

In [4]:
# get_client()
cliente = get_client()
cliente

In [17]:
# ingesta_inicial(client, limite)
datos_his = ingesta_inicial(cliente, 300000)
#datos_his

In [18]:
type(datos_his)

bytes

In [19]:
# get_s3_resource()
s3_resource = get_s3_resource()
type(s3_resource)

boto3.resources.factory.s3.ServiceResource

In [20]:
bucket = "mapreduce1ajgc"
bucket_path = "output/"
# bucket_path_hist = "ingestion/initial/historic-inspections-"
# bucket_path_cons = "ingestion/consecutive/consecutive-inspections-"

In [21]:
# guardar_ingesta(bucket, bucket_path, data)
guardar_ingesta(bucket, bucket_path, datos_his)

# guardar_ingesta(bucket, bucket_path_hist, datos_his)

Ingesta guardada


In [32]:
# ingesta_consecutiva(client, fecha, limite)
#x = ingesta_consecutiva(cliente, '2021-01-01', 2000)
#x

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2472264,AVENUE TAVERN,AVENUE TAVERN,1242927,Restaurant,Risk 1 (High),2914-2916 N BROADWAY,CHICAGO,IL,60657,2021-01-04T00:00:00.000,Canvass,Out of Business,41.935358121993126,-87.64438916677103,"{'latitude': '-87.64438916677103', 'longitude'...",NaN
1,2472249,KIMCHI POP BY CHEF SON,KIMCHI POP,2627373,Restaurant,Risk 1 (High),939 N ASHLAND AVE,CHICAGO,IL,60622,2021-01-04T00:00:00.000,Non-Inspection,No Entry,41.89907980640727,-87.66722805715766,"{'latitude': '-87.66722805715766', 'longitude'...",NaN
2,2472239,"SU NUEVA CASA DEL GALLO BRAVO, INC.","SU NUEVA CASA DEL GALLO BRAVO, INC.",2308713,Restaurant,Risk 1 (High),3714 W LAWRENCE AVE,CHICAGO,IL,60625,2021-01-04T00:00:00.000,Canvass,Fail,41.968423847600164,-87.72135178109974,"{'latitude': '-87.72135178109974', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
3,2472243,L & P LIQUORS INC,L & P LIQUORS,2757264,Restaurant,Risk 2 (Medium),4001 W LAWRENCE AVE,CHICAGO,IL,60630,2021-01-04T00:00:00.000,License,Fail,41.96813178484842,-87.72808675595967,"{'latitude': '-87.72808675595967', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
4,2464206,STARBUCKS COFFEE #2484,STARBUCKS COFFEE #2484,1139807,Restaurant,Risk 2 (Medium),111 E CHESTNUT ST,CHICAGO,IL,60611,2021-01-04T00:00:00.000,Canvass,Pass w/ Conditions,41.89826161908875,-87.62516469194003,"{'latitude': '-87.62516469194003', 'longitude'...","1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,2484892,MOBIL,MOBIL,2767226,Grocery Store,Risk 3 (Low),6000 S WESTERN AVE,CHICAGO,IL,60636,2021-02-18T00:00:00.000,License,Pass,41.78463932394361,-87.68393594077298,"{'latitude': '-87.68393594077298', 'longitude'...",36. THERMOMETERS PROVIDED & ACCURATE - Comment...
1598,2484872,NICK'S ON NORTH,NICK'S ON NORTH,2670594,Restaurant,Risk 1 (High),3323 W NORTH AVE,CHICAGO,IL,60647,2021-02-18T00:00:00.000,Canvass Re-Inspection,Pass,41.90995972342911,-87.71042949751607,"{'latitude': '-87.71042949751607', 'longitude'...",NaN
1599,2484889,CAFE COLAO,CAFE COLAO,2500992,Restaurant,Risk 1 (High),2638 W DIVISION ST,CHICAGO,IL,60622,2021-02-18T00:00:00.000,Canvass Re-Inspection,Pass,41.903064337614886,-87.69352967027304,"{'latitude': '-87.69352967027304', 'longitude'...",NaN
1600,2484869,"LITTLE HANDS LEARNING CENTER ACADEMY, INC","LITTLE HANDS LEARNING CENTER ACADEMY, INC",2216116,1023 CHILDERN'S SERVICES FACILITY,Risk 1 (High),10126 S WESTERN AVE,CHICAGO,IL,60643,2021-02-18T00:00:00.000,License,Pass,41.70921709772374,-87.68185356852545,"{'latitude': '-87.68185356852545', 'longitude'...",39. CONTAMINATION PREVENTED DURING FOOD PREPAR...


In [34]:
# ingesta_consecutiva(client, fecha, limite)
datos_cons = ingesta_consecutiva(cliente, '2021-01-01', 2000)
type(datos_cons)

bytes

In [35]:
bucket = "mapreduce1ajgc"
bucket_path = "output/consecutive/"

In [36]:
# guardar_ingesta(bucket, bucket_path, data)
guardar_ingesta(bucket, bucket_path, datos_cons)

# guardar_ingesta(bucket, bucket_path_cons, datos_cons)

Ingesta guardada
